In [2]:
import pickle
import os
import sys
sys.path.insert(0, '../olympus')
sys.path.insert(0, '../graphnet/src')
sys.path.insert(0, '../gnn_testbed')
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="0.1"

import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from tqdm.auto import tqdm

import numpy as np
import seaborn as sns
import random
from jax import random
from jax import numpy as jnp

import torch
import torch_cluster
import torch_geometric
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from nemesis.event_generation.detector import make_line, generate_noise, Detector, make_triang
from nemesis.plotting import plot_event, plot_events, plot_confusion
from nemesis.data_handling.utils import event_labelling
from nemesis.node_features.feature_generation import generate_features
from nemesis.evaluation.evaluation import model_evaluation, energy_division_loaders, energy_evaluation
from nemesis.evaluation.utils import count_parameters
from nemesis.models.train import train_model
from nemesis.models.gnns import Dynamic_class, DynEdge_modified, Dyn_own

from torch.nn import Linear, Identity, ReLU, Softmax, Dropout, LeakyReLU
import torch.nn.functional as F
from torch_geometric.nn import knn_graph, TAGConv, global_mean_pool, global_max_pool, BatchNorm,  global_add_pool, EdgeConv, DynamicEdgeConv

from graphnet.models.gnn.dynedge import DynEdge
from graphnet.components.layers import DynEdgeConv
from graphnet.models.gnn.gnn import GNN
from graphnet.models.utils import calculate_xyzt_homophily, calculate_xyz_homophily_POne

from multiprocessing import Pool
%matplotlib inline


In [3]:
outpath = "."
plt.rcParams['figure.dpi'] = 100
if torch.cuda.is_available():
    print('CUDA')
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print('CPU')

CUDA


In [4]:
def load_test_events(v_x):
    path_to_events = "/dss/pone/pone_events"
    det, cascades, cascade_records = pickle.load(open(os.path.join(path_to_events, f"cascades1500_vx{v_x}.pickle"), "rb"))
    det, tracks, track_records = pickle.load(open(os.path.join(path_to_events, f"tracks1500_vx{v_x}.pickle"), "rb"))
    det, stracks, strack_records = pickle.load(open(os.path.join(path_to_events, f"starting_tracks1500_vx{v_x}.pickle"), "rb"))
    
    return det, cascades, cascade_records, tracks, track_records, stracks, strack_records

In [5]:
def generate_loaders_vx(v_x):
    R=75
    det, cascades, cascade_records, tracks, track_records, stracks, strack_records = load_test_events(v_x)
    cascade_labels, track_labels, strack_labels = event_labelling(track_records, strack_records, cascade_records, det_hull=(R,1000))
    all_events = cascades + tracks + stracks
    all_records = cascade_records + track_records + strack_records
    all_labels = cascade_labels + track_labels + strack_labels
    data_array_test = generate_features(det, all_events, all_labels, k=15)
    pickle.dump(data_array_test,open(f"/dss/pone/pone_events/features_arrays/test_data_array/vx_data_arrays/data_array_{4.5}k_k{15}_R{R}_v_x{v_x}.pickle", "wb"))
    loader = DataLoader(data_array_test, batch_size=200, shuffle=False)
    pickle.dump(data_array_test,open(f"/dss/pone/pone_events/features_arrays/test_data_array/vx_loaders/test_loader_k{15}_R{R}_v_x{v_x}.pickle", "wb"))
    print(f"speed {v_x} done")
    return loader

In [6]:
v_x_all = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1]
for v_x in v_x_all:
    generate_loaders_vx(v_x)
    
if __name__ == "__main__":
    with Pool(10) as p:
        loader = p.map(generate_loaders_vx, v_x_all)


  0%|          | 0/4500 [00:00<?, ?it/s]

speed 0.005 done


  0%|          | 0/4500 [00:00<?, ?it/s]

speed 0.01 done


  0%|          | 0/4500 [00:00<?, ?it/s]

...

  0%|          | 0/4500 [00:00<?, ?it/s]

speed 0.095 done


  0%|          | 0/4500 [00:00<?, ?it/s]

speed 0.1 done


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

In [ ]:
def generate_loaders_vx(v_x):
    R=100
    det, cascades, cascade_records, tracks, track_records, stracks, strack_records = load_test_events(v_x)
    cascade_labels, track_labels, strack_labels = event_labelling(track_records, strack_records, cascade_records, det_hull=(R,1000))
    all_events = cascades + tracks + stracks
    all_records = cascade_records + track_records + strack_records
    all_labels = cascade_labels + track_labels + strack_labels
    data_array_test = generate_features(det, all_events, all_labels, k=15)
    pickle.dump(data_array_test,open(f"/dss/pone/pone_events/features_arrays/test_data_array/vx_data_arrays/data_array_{4.5}k_k{15}_R{R}_v_x{v_x}.pickle", "wb"))
    loader = DataLoader(data_array_test, batch_size=200, shuffle=False)
    pickle.dump(data_array_test,open(f"/dss/pone/pone_events/features_arrays/test_data_array/vx_loaders/test_loader_k{15}_R{R}_v_x{v_x}.pickle", "wb"))
    print(f"speed {v_x} done")
    return loader

In [ ]:
v_x_all = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1]
for v_x in v_x_all:
    generate_loaders_vx(v_x)
    
if __name__ == "__main__":
    with Pool(10) as p:
        loader = p.map(generate_loaders_vx, v_x_all)